In [1]:
import tensorflow as tf
import keras
from keras import utils
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
tqdm.pandas()
# keras set seed
utils.set_random_seed(1234)

2023-05-07 16:00:57.381335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 16:00:58.033101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load the data
DATA_FOLDER = "/mnt/d/sign-lang-data/"
train = pd.read_csv(os.path.join(DATA_FOLDER, "train.csv"))

In [3]:
s_train = train.sample(2000)
dict_sign = {}
for i, sign in enumerate(s_train["sign"].unique()):
    dict_sign[sign] = i

In [4]:
s_train

,path,participant_id,sequence_id,sign
77681,train_landmark_files/49445/4179379151.parquet,49445,4179379151,puzzle
32268,train_landmark_files/2044/2321535930.parquet,2044,2321535930,talk
73619,train_landmark_files/16069/4015771323.parquet,16069,4015771323,chair
92262,train_landmark_files/32319/908757938.parquet,32319,908757938,food
94183,train_landmark_files/29302/987765989.parquet,29302,987765989,pen
...,...,...,...,...
77415,train_landmark_files/27610/416789928.parquet,27610,416789928,wait
32027,train_landmark_files/29302/2311259378.parquet,29302,2311259378,pig
74394,train_landmark_files/4718/404775947.parquet,4718,404775947,stay
58468,train_landmark_files/30680/3392431752.parquet,30680,3392431752,man


In [5]:
max_nb_frames = 100
def preprocess(parquet_path):
    df = pd.read_parquet(os.path.join(DATA_FOLDER,parquet_path), engine="pyarrow", columns=["frame", "x", "y", "z"])

    df = df.fillna(0)
    # create a sequence as a list of frames (each frame is a np.array)
    frames = []
    for name, timestep in df.groupby("frame"):
        frame = np.stack(timestep[["x", "y", "z"]].values)
        frames.append(frame)

    sequence = np.reshape(np.stack(frames), (len(frames),1629))

    if len(sequence) > max_nb_frames:
        sequence = sequence[:max_nb_frames]
    else:
        sequence = np.concatenate((sequence, np.zeros((max_nb_frames - len(sequence), 1629))))

    return sequence

"""def get_label(sequence, sign, dict_):
    labels = []
    for i in range(len(sequence)):
        labels.append(utils.to_categorical(dict_[sign], num_classes=len(s_train["sign"].unique())))
    return np.stack(labels)"""

def get_label(sequence, sign, dict_):
    return utils.to_categorical(dict_[sign], num_classes=len(s_train["sign"].unique()))

In [6]:
# Preprocess the data
# X
s_train["sequence"] = s_train["path"].progress_apply(preprocess)
# Y
# get max length of sequence
#max_length = s_train["sequence"].progress_apply(lambda x: len(x)).max()
# complete the sequence with zeros
#s_train["sequence"] = s_train["sequence"].progress_apply(lambda x: np.concatenate((x, np.zeros((max_length - len(x), 1629)))))

s_train["label"] = s_train.progress_apply(lambda x: get_label(x["sequence"], x["sign"], dict_sign), axis=1)
#s_train["label"] =


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

In [7]:
from keras import Input
from keras.layers import Dense, LSTM, Masking
# Inisialize the model from custom class
from custom_model.first import CustomModel
batch_size = 10
timesteps = 100
features = 1629
nb_classes = len(s_train["sign"].unique())

model = CustomModel(batch_size, timesteps, features, nb_classes)
# model = CustomModel( input_layer, output_layer)
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['categorical_accuracy'], sample_weight_mode='temporal')

2023-05-07 16:02:12.104263: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-07 16:02:12.238132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-07 16:02:12.238222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-07 16:02:12.244654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-07 16:02:12.244722: I tensorflow/compile

In [8]:
# Train the model
x_train = np.stack(s_train["sequence"].values)
y_train = np.stack(s_train["label"].values)
del s_train
del train
del dict_sign
# do split with sklearn
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
del x_train
del y_train

In [ ]:
metrics = model.fit(X_train, Y_train , validation_data=(X_test, Y_test), epochs=100, batch_size=10)

2023-05-07 16:02:16.760880: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1042560000 exceeds 10% of free system memory.
2023-05-07 16:02:17.474316: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1042560000 exceeds 10% of free system memory.


Epoch 1/100


2023-05-07 16:02:20.781589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-07 16:02:20.980380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-07 16:02:21.527804: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f67a7748550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-07 16:02:21.527859: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-05-07 16:02:21.698989: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-07 16:02:21.825173: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


160/160 [==============================] - 8s 25ms/step - loss: 5.5401 - categorical_accuracy: 0.0044 - val_loss: 5.5235 - val_categorical_accuracy: 0.0050
Epoch 2/100
160/160 [==============================] - 2s 15ms/step - loss: 5.5216 - categorical_accuracy: 0.0081 - val_loss: 5.5234 - val_categorical_accuracy: 0.0075
Epoch 3/100
160/160 [==============================] - 2s 15ms/step - loss: 5.5090 - categorical_accuracy: 0.0075 - val_loss: 5.5314 - val_categorical_accuracy: 0.0075
Epoch 4/100
160/160 [==============================] - 2s 15ms/step - loss: 5.5005 - categorical_accuracy: 0.0075 - val_loss: 5.5299 - val_categorical_accuracy: 0.0075
Epoch 5/100
160/160 [==============================] - 2s 14ms/step - loss: 5.4913 - categorical_accuracy: 0.0050 - val_loss: 5.5337 - val_categorical_accuracy: 0.0075
Epoch 6/100
160/160 [==============================] - 2s 15ms/step - loss: 5.4846 - categorical_accuracy: 0.0088 - val_loss: 5.5390 - val_categorical_accuracy: 0.0075
Epoc

In [ ]:
# Plot the metrics
plt.plot(metrics.history['categorical_accuracy'])
plt.plot(metrics.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'])


In [ ]:
plt.plot(metrics.history['loss'])
plt.plot(metrics.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])